In [25]:
import os

train_dir = r"plant_disease_dataset_10_classes_split/plant_disease_dataset_10_classes_split/train"

test_dir = r"plant_disease_dataset_10_classes_split/plant_disease_dataset_10_classes_split/test"

train_classes = os.listdir(train_dir)
test_classes = os.listdir(test_dir)
print(f"Number of classes: {len(train_classes)}")
print(train_classes)

print(f"Number of classes: {len(test_classes)}")
print(test_classes)

Number of classes: 10
['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']
Number of classes: 10
['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']


Transformation of Train dataset

In [26]:
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

train_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

train_dataset = ImageFolder(train_dir, transform = train_transform)

In [27]:
from collections import Counter

train_labels = train_dataset.targets

count_per_classes = Counter(train_labels)

print(f"Total Sample in train_dataset:", len(train_dataset))
print("="*25)

for i, class_name in enumerate(train_dataset.classes):
    print(f"{class_name:<10} -> {count_per_classes[i]}")

Total Sample in train_dataset: 12006
Tomato_Bacterial_spot -> 1595
Tomato_Early_blight -> 750
Tomato_Late_blight -> 1431
Tomato_Leaf_Mold -> 714
Tomato_Septoria_leaf_spot -> 1328
Tomato_Spider_mites_Two_spotted_spider_mite -> 1257
Tomato__Target_Spot -> 1053
Tomato__Tomato_YellowLeaf__Curl_Virus -> 2406
Tomato__Tomato_mosaic_virus -> 279
Tomato_healthy -> 1193


Transformation of Test Dataset

In [28]:
test_transform = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_dataset = ImageFolder(test_dir, transform=test_transform)

In [29]:
test_labels = test_dataset.targets
count_per_classes = Counter(test_labels)

print(f"The total Sample in the test dataset:", len(test_dataset))
print("="*25)

for i, class_name in enumerate(test_dataset.classes):
    print(f"{class_name:<10} -> {count_per_classes[i]}")

The total Sample in the test dataset: 4005
Tomato_Bacterial_spot -> 532
Tomato_Early_blight -> 250
Tomato_Late_blight -> 478
Tomato_Leaf_Mold -> 238
Tomato_Septoria_leaf_spot -> 443
Tomato_Spider_mites_Two_spotted_spider_mite -> 419
Tomato__Target_Spot -> 351
Tomato__Tomato_YellowLeaf__Curl_Virus -> 802
Tomato__Tomato_mosaic_virus -> 94
Tomato_healthy -> 398


Splitting the train data into Train and Val data

In [30]:
import torch
from torch.utils.data import random_split

torch.manual_seed = 32

total_size = len(train_dataset)

train_size = int(0.70*len(train_dataset))
val_size = total_size - train_size

train_ds, val_ds = random_split(train_dataset, [train_size, val_size])

print("The sample in the train dataset:", len(train_dataset))
print("The size of train dataset:", train_size)
print("The size of val dataset:", val_size)


The sample in the train dataset: 12006
The size of train dataset: 8404
The size of val dataset: 3602


Using DataLoader

In [31]:
from torch.utils.data import DataLoader

batch_size = 32

train_loader = DataLoader(train_ds, batch_size = batch_size, shuffle = True)

val_loader = DataLoader(val_ds, batch_size = batch_size, shuffle = False)

In [32]:
import torch.nn as nn
import torch
import torch.nn.functional as F

def accuracy(outputs, labels):
    preds = torch.argmax(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  
        loss = F.cross_entropy(out, labels)  
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    
        loss = F.cross_entropy(out, labels)   
        acc = accuracy(out, labels)           
        return {'val_loss': loss.detach(), 'val_acc': acc.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result, is_best=False):
        msg = (
            f"Epoch {epoch} | "
            f"Train Acc: {result['train_acc']:.4f} | "
            f"Train Loss: {result['train_loss']:.4f} | "
            f"Val Loss: {result['val_loss']:.4f} | "
            f"Val Acc: {result['val_acc']:.4f}"
        )
        if is_best:
            msg += "  <-- Saving best model "
        print(msg)

In [33]:
import torchvision.models as models

class ResNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()

        self.network = models.resnet50(pretrained=True)

        
        num_ftrs = self.network.fc.in_features

       
        self.network.fc = nn.Sequential(
            nn.Dropout(p=0.2),               
            nn.Linear(num_ftrs, len(train_dataset.classes))
        )

    def forward(self, xb):
        return self.network(xb)

Making changes in the Classification head

In [34]:
class ResNet(ImageClassificationBase):
    def __init__(self, num_classes):
        super().__init__()
        self.network = models.resnet50(weights= None)
        
        num_ftrs = self.network.fc.in_features

        self.network.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4), 
            nn.Linear(512, num_classes)
        )

    def forward(self, xb):
        return self.network(xb)

In [35]:
num_class = 10
model = ResNet(num_class)
state_dict = torch.load('best_model.pth', map_location = 'cpu')
model.load_state_dict(state_dict , strict = True )
model.eval()

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "network.conv1.weight", "network.bn1.weight", "network.bn1.bias", "network.bn1.running_mean", "network.bn1.running_var", "network.layer1.0.conv1.weight", "network.layer1.0.bn1.weight", "network.layer1.0.bn1.bias", "network.layer1.0.bn1.running_mean", "network.layer1.0.bn1.running_var", "network.layer1.0.conv2.weight", "network.layer1.0.bn2.weight", "network.layer1.0.bn2.bias", "network.layer1.0.bn2.running_mean", "network.layer1.0.bn2.running_var", "network.layer1.0.conv3.weight", "network.layer1.0.bn3.weight", "network.layer1.0.bn3.bias", "network.layer1.0.bn3.running_mean", "network.layer1.0.bn3.running_var", "network.layer1.0.downsample.0.weight", "network.layer1.0.downsample.1.weight", "network.layer1.0.downsample.1.bias", "network.layer1.0.downsample.1.running_mean", "network.layer1.0.downsample.1.running_var", "network.layer1.1.conv1.weight", "network.layer1.1.bn1.weight", "network.layer1.1.bn

In [140]:
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


model = ResNet(num_classes = 10)
model = model.to(device)

Using device: cpu


In [141]:
import torch.optim as optim
opt_func = torch.optim.SGD

optimizer = opt_func(
    model.parameters(),
    lr=1e-3,
    momentum=0.9,
    weight_decay=1e-4
)

In [142]:
def fit(epochs, model, train_loader, val_loader, optimizer, device, checkpoint_path="best_model.pth"):
    history = []
    best_val_acc = 0.0 

    for epoch in range(epochs):
      
        model.train()
        train_losses = []
        train_correct = 0
        train_total = 0

        for batch in train_loader:
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)


            loss = F.cross_entropy(outputs, labels)
            train_losses.append(loss)

     
            _, preds = torch.max(outputs, dim=1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        train_loss = torch.stack(train_losses).mean().item()
        train_acc = train_correct / train_total

        
        model.eval()
        with torch.no_grad():
            val_outputs = []
            for batch in val_loader:
                images, labels = batch
                images = images.to(device)
                labels = labels.to(device)

                out = model.validation_step((images, labels))
                val_outputs.append(out)

            result = model.validation_epoch_end(val_outputs)
            result['train_loss'] = train_loss
            result['train_acc'] = train_acc

        
        val_acc = result['val_acc']
        is_best = val_acc > best_val_acc

        
        model.epoch_end(epoch + 1, result, is_best=is_best)
        history.append(result)

       
        if is_best:
            best_val_acc = val_acc
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "val_loss": result['val_loss'],
                    "val_acc": result['val_acc'],
                },
                checkpoint_path,
            ) 

    return history

In [143]:
epochs = 5
history = fit(epochs, model, train_loader, val_loader, optimizer,
              device, checkpoint_path="best_model.pth")

Epoch 1 | Train Acc: 0.7750 | Train Loss: 0.7549 | Val Loss: 0.2068 | Val Acc: 0.9528  <-- Saving best model 
Epoch 2 | Train Acc: 0.9599 | Train Loss: 0.1635 | Val Loss: 0.1168 | Val Acc: 0.9685  <-- Saving best model 
Epoch 3 | Train Acc: 0.9832 | Train Loss: 0.0788 | Val Loss: 0.0768 | Val Acc: 0.9804  <-- Saving best model 
Epoch 4 | Train Acc: 0.9919 | Train Loss: 0.0443 | Val Loss: 0.0673 | Val Acc: 0.9823  <-- Saving best model 
Epoch 5 | Train Acc: 0.9968 | Train Loss: 0.0258 | Val Loss: 0.0692 | Val Acc: 0.9787


Saving the model using Joblib

In [173]:
import joblib

joblib.dump(model, 'leaf_disease_model.joblib')


['leaf_disease_model.joblib']

Adding Generative AI Compoments 

In [174]:
!pip install transformers torch


[notice] A new release of pip available: 22.2.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [171]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

In [168]:
model_name = "gpt2"

In [172]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading weights: 100%|██████████| 148/148 [00:01<00:00, 109.24it/s, Materializing param=transformer.wte.weight]             
GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
generator = pipeline (
    "text-generation",
    model = model_name,
    tokenizer = model_name,
    torch_dtype = 'auto'
)

prompt = f"""
### Instruction:
You are a professional plant pathologist. A deep learning model has identified a leaf disease in a crop. 
Provide a concise, expert report for a farmer.

### Disease Detected: 
{predicted_disease}

### Report Requirements:
1. Symptoms: Briefly describe how to confirm this.
2. Immediate Action: What should the farmer do today?
3. Organic Treatment: List two natural remedies.
4. Prevention: How to avoid this in the next season.

### Expert Advice:
"""

max_token = 300
outputs = generator(prompt, max_new_tokens=max_token)

print(outputs[0]["generated_text"])

In [2]:
import torch
checkpoint = torch.load("best_model.pth", map_location=device)
print(type(checkpoint))


NameError: name 'device' is not defined